In [1]:
import json
import graphene

print(graphene.__version__)

def unorder(ordered_dict):
    return json.dumps(ordered_dict, indent=4)

def run(query, request):
    schema = graphene.Schema(query=query)
    result = schema.execute(request)
    if result.errors:
        raise(result.errors)
    print( unorder(result.data) )

2.0.1


In [2]:
class Query(graphene.ObjectType):
    hi = graphene.String()
    
    hello = graphene.String(name=graphene.String(default_value="stranger"))

    def resolve_hi(self, info):
        return "Hi! you look so pretty today"
    
    def resolve_hello(self, info, name):
        return 'Hello ' + name

In [3]:
run(Query, '{ hi }')
run(Query, '{ hello }')
run(Query, '{ hello (name: "MY_NAME") }')

{
    "hi": "Hi! you look so pretty today"
}
{
    "hello": "Hello stranger"
}
{
    "hello": "Hello MY_NAME"
}


In [4]:
class User(graphene.ObjectType):
    id = graphene.Int()
    username = graphene.String()

In [5]:
class Query1(graphene.ObjectType):
    
    user = graphene.Field(User)

    def resolve_user(self, info):
        return User(id=1, username='abcd')

In [6]:
run(Query1, '{ user { id } }')
run(Query1, '{ user { id username } }')

{
    "user": {
        "id": 1
    }
}
{
    "user": {
        "id": 1,
        "username": "abcd"
    }
}


In [7]:
class Classroom(graphene.ObjectType):
    id = graphene.Int()
    name = graphene.String()
    students = graphene.Field( graphene.List( User ) )
    
    def resolve_students(self, info):
        print('class_id:', self.id)
        users = [
            User(id=i, username='NO.' + str(i)) 
            for i in range(100 * self.id, 100 * self.id + 3)
        ]
        return users
    
class Query2(graphene.ObjectType):
    
    classrooms = graphene.Field( graphene.List( Classroom ) )

    def resolve_classrooms(self, info):
        return [
            Classroom(id = i, name = f'Classroom {i}')
            for i in range(1, 3)
        ]

run(Query2, '{ classrooms { name students { username } } }')

class_id: 1
class_id: 2
{
    "classrooms": [
        {
            "name": "Classroom 1",
            "students": [
                {
                    "username": "NO.100"
                },
                {
                    "username": "NO.101"
                },
                {
                    "username": "NO.102"
                }
            ]
        },
        {
            "name": "Classroom 2",
            "students": [
                {
                    "username": "NO.200"
                },
                {
                    "username": "NO.201"
                },
                {
                    "username": "NO.202"
                }
            ]
        }
    ]
}
